In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [2]:
from transformers import BertTokenizer, BertForTokenClassification, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import TextDataset

In [ ]:
!pip install python-docx

In [4]:
import json
# Read the JSON file
path = "/content/drive/MyDrive/model/B_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

# Assuming the dataset is a list of dictionaries

# Define a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create an instance of CustomDataset
custom_dataset = CustomDataset(dataset)

# Create a DataLoader
batch_size = 1
train_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [5]:
key = []
for sample in train_dataloader:
    key.append(sample['entities'])

In [6]:
entities = []
for substr in key:
    tokens = substr[0].split()
    entities.extend(tokens)

In [ ]:
entities

In [8]:
key = set(entities)
print(key)
print(len(key))
num_ner_labels = len(key)

{'O', 'I-DATE', 'B-TIME', 'I-DUR', 'B-DUR', 'B-DATE', 'I-TASK', 'B-TASK', 'I-TIME'}
9


In [9]:
l = []
for sample in train_dataloader:
    l.extend(sample['intent'])

In [10]:
l = set(l)
print(l)
print(len(l))
num_intent_labels=len(l)

{"'Set Timer'", "'Schedule Meeting'", "'Set Alarm'", "'Schedule Appointment'", "'Set Reminder'"}
5


In [11]:
# Load the BERT tokenizer and models for token classification and sequence classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
ner_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_ner_labels)
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent_labels)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenizing the dataset

Maybe I should try to set trincation and padding to False.

In [12]:
tokenized_inputs = []

max_sequence_length = 0  # Initialize maximum sequence length

for batch in train_dataloader:
    text = batch['text']
    batch_tokenized = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    input_ids = batch_tokenized['input_ids']
    attention_mask = batch_tokenized['attention_mask']
    tokenized_inputs.append((input_ids, attention_mask))

    # Update maximum sequence length
    max_sequence_length = max(max_sequence_length, input_ids.shape[1])

# Pad sequences to the maximum sequence length
for i in range(len(tokenized_inputs)):
    input_ids, attention_mask = tokenized_inputs[i]
    pad_length = max_sequence_length - input_ids.shape[1]
    padded_input_ids = torch.nn.functional.pad(input_ids, (0, pad_length), value=tokenizer.pad_token_id)
    padded_attention_mask = torch.nn.functional.pad(attention_mask, (0, pad_length), value=0)  # Assuming 0 for padding mask
    tokenized_inputs[i] = (padded_input_ids, padded_attention_mask)

# Concatenate input IDs and attention masks separately
input_ids = torch.cat([tensor[0] for tensor in tokenized_inputs], dim=0)
attention_mask = torch.cat([tensor[1] for tensor in tokenized_inputs], dim=0)



In [13]:
input_ids

tensor([[  101,  1000,  6134,  2019,  6098,  2005,  2279,  5958,  2012,  1023,
          2572,  1012,  1000,   102,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1000, 10825,  2033,  2055,  1996,  3116,  2012,  1017,  7610,
          4826,  1012,  1000,   102,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1000, 10825,  2033,  2000,  4965, 26298,  4826,  2851,  1012,
          1000,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1000,  2275,  2019,  8598,  2005,  1021,  2572,  1012,  1000,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1000, 10825,  2033,  2000,  2655,  2198,  1999,  2382,  2781,
          1012,  1000,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1000,  2275,  1037, 25309,  2005,  2184,  2781,  1012,  1000,
           

In [23]:
# task -> B-TASK
# date -> B-DATE
# duration -> B-DUR
# days -> B_DAY
# time -> B-TIME
# specific_time
encoded_ner_labels = []
label_ner_map = {'O': 0, 'B-DATE': 1, 'I-DATE': 2, 'B-TIME': 3, 'I-TIME': 4, 'B-TASK': 5, 'I-TASK': 6, 'B-DUR': 7, 'I-DUR': 8}
for sample in train_dataloader:
    encoded_ner_labels.extend([label_ner_map[label] for label in sent_label.split()] for sent_label in sample['entities'])
encoded_ner_labels

[[0, 0, 0, 5, 6, 1, 3],
 [0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 3, 4],
 [0, 0, 0, 0, 0, 1, 2, 0, 3, 4],
 [0, 0, 0, 0, 0, 0, 5, 0, 7, 8],
 [0, 0, 0, 0, 1, 2, 0, 3, 4],
 [0, 0, 0, 0, 0, 3, 5],
 [0, 0, 0, 0, 7, 8],
 [0, 0, 0, 0, 0, 0, 3, 4, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 5, 6, 0, 3, 4, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 3, 4],
 [0, 0, 0, 0, 3, 4],
 [0, 0, 0, 0, 1, 2, 3],
 [0, 0, 0, 5, 6, 0, 7, 8]]

In [41]:
for i in train_dataloader:
    print(i['entities'])

['O O O O B-TIME I-TIME']
['O O O O B-TASK I-TASK O B-TIME I-TIME O B-DATE']
['O O O O B-DATE I-DATE B-TIME']
['O O O O O O B-TASK O B-DUR I-DUR']
['O O O O O O B-DATE I-DATE I-DATE I-DATE I-DATE O B-TIME I-TIME']
['O O O O O O O O O O B-DATE']
['O O O O O O B-TIME I-TIME B-DATE']
['O O O B-TASK I-TASK O B-DUR I-DUR']
['O O O O B-DUR I-DUR']
['O O O O O B-DATE I-DATE O B-TIME I-TIME']
['O O O B-TASK I-TASK B-DATE B-TIME']
['O O O O O O O O B-DATE I-DATE I-DATE I-DATE O B-TIME I-TIME']
['O O O O O B-TIME B-TASK']
['O O O O B-DATE I-DATE O B-TIME I-TIME']


In [40]:
from sklearn.preprocessing import LabelEncoder
sample_entities = []
for sample in train_dataloader:
    sample_entities.extend(sample['entities'])

# Flatten the list of strings into a single list of tokens
all_entities = [entity for sample in sample_entities for entity in sample.split()]

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder to all unique entity labels
label_encoder.fit(all_entities)

# Encode entities for each sample in sample_entities
encoded_entities = [
    label_encoder.transform(entity.split()).tolist() for entity in sample_entities
]

print(encoded_entities)


[[8, 8, 8, 8, 1, 5], [8, 8, 8, 8, 0, 4, 8, 3, 7], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0], [8, 8, 8, 8, 8, 3, 2], [8, 8, 8, 8, 8, 8, 3, 7, 0], [8, 8, 8, 8, 8, 8, 2, 8, 1, 5], [8, 8, 8, 2, 6, 0, 3], [8, 8, 8, 2, 6, 8, 1, 5], [8, 8, 8, 8, 8, 8, 0, 4, 4, 4, 4, 8, 3, 7], [8, 8, 8, 8, 3, 7], [8, 8, 8, 8, 8, 8, 8, 8, 0, 4, 4, 4, 8, 3, 7], [8, 8, 8, 8, 8, 0, 4, 8, 3, 7], [8, 8, 8, 8, 2, 6, 8, 3, 7, 8, 0], [8, 8, 8, 8, 0, 4, 3]]


In [ ]:
intent_labels = list(l)

encoded_intent_labels = label_encoder.fit_transform(intent_labels)

In [ ]:
class TokenizedDataset(Dataset):
    def __init__(self, tokenized_inputs, encoded_ner_labels, encoded_intent_labels):
        self.tokenized_inputs = tokenized_inputs
        self.encoded_ner_labels = encoded_ner_labels
        self.encoded_intent_labels = encoded_intent_labels

    def __len__(self):
        return len(self.tokenized_inputs)

    def __getitem__(self, idx):
        inputs = self.tokenized_inputs[idx]
        ner_labels = self.encoded_ner_labels[idx]
        intent_labels = self.encoded_intent_labels[idx]

        return {
            'inputs': inputs,
            'ner_labels': ner_labels,
            'intent_labels': intent_labels
        }

In [ ]:
cust_dataset = CustomDataset(tokenized_inputs, encoded_ner_labels, encoded_intent_labels)
data_loader = DataLoader(cust_dataset, batch_size=1, shuffle=True)

In [ ]:
ner_model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
intent_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
for sample in train_dataloader:
    print(sample['entities'])

{'task': ['cooking'], 'duration': ['1 hour']}
{'time': ['3 PM'], 'date': ['tomorrow']}
{'duration': ['10 minutes']}
{'task': ['5-minute meditation session'], 'duration': ['5 minutes']}
{'task': ['webinar'], 'time': ['2 PM'], 'duration': ['2 days']}
{'date': ['10th of next month'], 'time': ['2 PM']}
{'task': ['20-minute workout session'], 'duration': ['20 minutes']}
{'task': ['buy groceries'], 'date': ['Saturday'], 'time': ['afternoon']}
{'task': ['study session'], 'duration': ['45 minutes']}
{'time': ['7:30 AM'], 'date': ['tomorrow']}
{'task': ['water the plants'], 'time': ['9 AM'], 'days': [('Tuesday',), ('Thursday',)]}
{'task': ['pay bills'], 'date': ['last day of the month']}
{'duration': ['30 minutes'], 'task': ['call John']}
{'task': ['buy groceries'], 'date': ['tomorrow'], 'time': ['morning']}
{'date': ['end of the month'], 'time': ['4:30 PM']}
{'date': ['next Monday'], 'time': ['morning'], 'specific_time': ['10:30']}
{'date': ['March 20th'], 'time': ['10:30 AM']}
{'time': ['2:30

In [ ]:
optimizer = torch.optim.Adam([
    {'params': ner_model.parameters()},
    {'params': intent_model.parameters()}
], lr=1e-5)  # Define optimizer for both models

num_epochs = 3  # Define number of epochs

for epoch in range(num_epochs):
    for batch in train_dataloader:  # Iterate over your dataset batches
        inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)

        # Forward pass for intent classification
        outputs_intent = intent_model(**inputs)
        intent_labels = batch['intent']
        intent_loss = torch.nn.CrossEntropyLoss()(outputs_intent.logits, intent_labels)

        # Forward pass for NER
        outputs_ner = ner_model(**inputs)
        ner_labels = batch['entities']
        ner_loss = torch.nn.CrossEntropyLoss()(outputs_ner.logits.view(-1, num_ner_labels), ner_labels.view(-1))

        # Total loss (you might adjust weights for different tasks)
        total_loss = ner_loss + intent_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
optimizer = torch.optim.Adam([
    {'params': ner_model.parameters()},
    {'params': intent_model.parameters()}
], lr=1e-5)  # Define optimizer for both models

num_epochs = 3  # Define number of epochs

for epoch in range(num_epochs):
    for batch in train_dataloader:  # Iterate over your dataset batches
        inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True)

        # Forward pass for intent classification
        outputs_intent = intent_model(**inputs)
        intent_labels = batch['intent']
        intent_loss = torch.nn.CrossEntropyLoss()(outputs_intent.logits, intent_labels)

        # Forward pass for NER
        outputs_ner = ner_model(**inputs)
        ner_labels = batch['entities']
        ner_loss = torch.nn.CrossEntropyLoss()(outputs_ner.logits.view(-1, num_ner_labels), ner_labels.view(-1))

        # Total loss (you might adjust weights for different tasks)
        total_loss = ner_loss + intent_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # No idea yet

Building the Multi-Task Architecture

In [ ]:
class MultiTaskBertWrapper(nn.Module):
    def __init__(self, ner_model, intent_model):
        super().__init__()
        self.ner_model = ner_model
        self.intent_model = intent_model

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None, task_type=None):
        if task_type = 'ner':
            self.ner_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        elif task_type = 'intent':
            self.intent_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        else:
            raise ValueError('Invalid task_type. Use "ner" or "intent"')

In [ ]:
model = MultiTaskBertWrapper(ner_model, intent_model)

In [ ]:
model

In [ ]:
params = list(model.ner_model.parameters()) + list(model.intent_model.parameters())
optimizer = torch.optim.AdamW(params, lr=1e-5)

ner_loss_function = torch.nn.CrossEntropyLoss()
intent_loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        inputs = batch['text']
        attention_masks = batch['attention_mask']
        labels_ner = batch['entities']
        labels_intent = batch['intent']

        optimizer.zero_grad()

        # Forward pass for NER task
        ner_outputs = model.ner_model(input_ids=inputs, attention_mask=attention_masks, labels=labels_ner, task_type='ner')
        ner_loss = ner_loss_function(ner_outputs.logits.view(-1, num_ner_labels), labels_ner.view(-1))

        ner_loss.backward()

        # Forward pass for Intent Classification task
        intent_outputs = model.intent_model(input_ids=inputs, attention_mask=attention_masks, labels=labels_intent, task_type='intent')
        intent_loss = intent_loss_function(intent_outputs.logits.view(-1, num_intent_labels), labels_intent.view(-1))

        intent_loss.backward()

        optimizer.step()